In [1]:
from numpy import array
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import ipynb
import keras
from sklearn.metrics import mean_squared_error
import pandas as pd

## Get GME and AMC Datasets

In [2]:
#Store variables for training
%store -r AMC_Price
%store -r AMC_Open
%store -r AMC_High
%store -r AMC_Low
%store -r AMC_Vol
%store -r AMC_Change

%store -r GME_Price
%store -r GME_Open
%store -r GME_High
%store -r GME_Low
%store -r GME_Vol
%store -r GME_Change

## Sequence for Splitting Datsets to Fit LSTM

In [3]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

## LSTM Prediction Model Training

In [4]:
def LSTM_Prediction_Model(train_price, train_open, train_high, train_low, train_volume, train_change): 
    data_price = train_price
    data_open = train_open
    data_high = train_high
    data_low = train_low
    data_vol = train_volume
    data_change = train_change

    # choose a number of time steps
    n_steps = 8
    # split into samples / reshape from x,y to x,y,z
    Xprice, y = split_sequence(data_price, n_steps)
    Xprice = Xprice.reshape((Xprice.shape[0], n_steps, 1))
    
    Xopen, y = split_sequence(data_open, n_steps)
    Xopen = Xopen.reshape((Xopen.shape[0], n_steps, 1))
    
    Xhigh, y = split_sequence(data_high, n_steps)
    Xhigh = Xhigh.reshape((Xhigh.shape[0], n_steps, 1))
    
    Xlow, y = split_sequence(data_low, n_steps)
    Xlow = Xlow.reshape((Xlow.shape[0], n_steps, 1))
    
    Xvol, y = split_sequence(data_vol, n_steps)
    Xvol = Xlow.reshape((Xvol.shape[0], n_steps, 1))
    
    Xchange, y = split_sequence(data_change, n_steps)
    Xchange = Xchange.reshape((Xchange.shape[0], n_steps, 1))
    
    #concatenate training data into one array (samples, steps, features)
    Xtraining = np.concatenate((Xprice, Xopen, Xhigh, Xlow, Xchange), axis = 2)

    n_filters=10
    fsize=5
    window_size=8
    n_features=5
    hour_filter = 8

    prediction_model = Sequential()
    layer1 = tf.keras.layers.Conv1D(n_filters, fsize, padding="same", activation="relu", input_shape=(window_size, n_features))
    layer2 = tf.keras.layers.Flatten()
    layer3 = tf.keras.layers.Dense(1000, activation='relu')
    layer4 = tf.keras.layers.Dense(100)
    layer5 = tf.keras.layers.Dense(1)
    prediction_model.add(layer1)
    prediction_model.add(layer2)
    prediction_model.add(layer3)
    prediction_model.add(layer4)
    prediction_model.add(layer5)
    prediction_model.compile(optimizer="adam", loss="mean_squared_error")
    prediction_model.fit(Xtraining, y, epochs=200, batch_size =32, verbose=1)
    return prediction_model

## Model Training and Saving

### Calling Training Function for Datasets

In [5]:
AMCModel = LSTM_Prediction_Model(AMC_Price, AMC_Open, AMC_High, AMC_Low, AMC_Vol, AMC_Change)
GMEModel = LSTM_Prediction_Model(GME_Price, GME_Open, GME_High, GME_Low, GME_Vol, GME_Change)

Epoch 1/200
80/80 [==============================] - 0s 1ms/step - loss: 333.2536
Epoch 2/200
80/80 [==============================] - 0s 1ms/step - loss: 7.5218
Epoch 3/200
80/80 [==============================] - 0s 1ms/step - loss: 6.1652
Epoch 4/200
80/80 [==============================] - 0s 1ms/step - loss: 2.6030
Epoch 5/200
80/80 [==============================] - 0s 1ms/step - loss: 0.8732
Epoch 6/200
80/80 [==============================] - 0s 1ms/step - loss: 0.3090
Epoch 7/200
80/80 [==============================] - 0s 1ms/step - loss: 0.3159
Epoch 8/200
80/80 [==============================] - 0s 1ms/step - loss: 0.3231
Epoch 9/200
80/80 [==============================] - 0s 1ms/step - loss: 0.1577
Epoch 10/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0729
Epoch 11/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0543
Epoch 12/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0464
Epoch 13/200
80/80 [===============

80/80 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 103/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0055
Epoch 104/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0022
Epoch 105/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0029
Epoch 106/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0029
Epoch 107/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0025
Epoch 108/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0024
Epoch 109/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0026
Epoch 110/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0031
Epoch 111/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0027
Epoch 112/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0034
Epoch 113/200
80/80 [==============================] - 0s 1ms/step - loss: 0.0034
Epoch 114/200
80/80 [=========

### Saving Models to be Called Elsewhere

In [7]:
AMCModel.save("AMCModel.h5")
GMEModel.save("GMEModel.h5")